In [249]:
import pandas as pd
import numpy as np
import coptpy as copt

In [250]:
result_full = pd.read_csv("data/附件3/result_temple_full.csv", header=1)
result_full

,年份,季度,地块名,地块id,黄豆,黑豆,红豆,绿豆,爬豆,小麦,...,黄心菜,芹菜,大白菜,白萝卜,红萝卜,榆黄菇,香菇,白灵菇,羊肚菌,种植上限
0,2023,1,A1,1,0.0,0.0,0.0,0.0,0.0,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
1,2023,1,A2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0
2,2023,1,A3,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0
3,2023,1,A4,4,72.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0
4,2023,1,A5,5,0.0,0.0,0.0,68.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,2030,2,E16,652,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.6
652,2030,2,F1,653,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
653,2030,2,F2,654,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
654,2030,2,F3,655,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6


In [251]:
# 用掩码来标记缺失值,只对缺失值进行规划
mask = ~np.isnan(result_full.iloc[:, 4:-1].values)
known_values = np.nan_to_num(result_full.iloc[:, 4:-1].values)

In [252]:
# 创建模型
env = copt.Envr()
model = env.createModel()

# X:种植量矩阵;D:种植决策矩阵
X = model.addVars(656, 41, vtype=copt.COPT.CONTINUOUS, lb=0)
D = model.addVars(656, 41, vtype=copt.COPT.BINARY)

Cardinal Optimizer v7.1.1. Build date Mar  4 2024
Copyright Cardinal Operations 2024. All Rights Reserved



In [253]:
# 约束1:只规划未知的值
for i in range(656):
    for j in range(41):
        if mask[i, j]:
            model.addConstr(X[i, j] == known_values[i, j])

# 约束2:种植量不能超过上限
for i in range(656):
    model.addConstr(
        copt.quicksum(X[i, j] for j in range(41)) <= result_full["种植上限"][i]
    )

# 约束3:如果种植,种植量不能低于下限  
for i in range(656):
    for j in range(41):
        model.addConstr(X[i, j] <= result_full["种植上限"][i] * D[i, j])
        model.addConstr(X[i, j] >= 0.2 * result_full["种植上限"][i] * D[i, j])

In [254]:
# 约束4:水稻约束
water_lands = [
    (
        i,
        result_full[(result_full["地块名"] == f"D{j}") & (result_full["季度"] == 1)][
            "种植上限"
        ].values[0],
    )
    for j in range(1, 9)
    for i in result_full[
        (result_full["地块名"] == f"D{j}") & (result_full["季度"] == 1)
    ].index[1:]
]

for x,u in water_lands:
    l = 0.2 * u
    for i in range(16,34):
        delta = model.addVar(vtype=copt.COPT.BINARY)
        model.addConstr(X[x, 15] <= u * (1 - delta))
        model.addConstr(X[x, i] <= u * delta)
    for i in range(34,37):
        delta = model.addVar(vtype=copt.COPT.BINARY)
        model.addConstr(X[x, 15] <= u * (1 - delta))
        model.addConstr(X[x+28, i] <= u * delta)  

In [255]:
# 约束5:重茬约束
for index, row in result_full[["地块名", "种植上限"]].drop_duplicates().iterrows():
    land_indices = result_full[result_full["地块名"] == row["地块名"]].index.tolist()

    u = row["种植上限"]
    l = 0

    for j in range(41):
        for i in range(len(land_indices) - 1):           
            idx_1 = land_indices[i]
            idx_2 = land_indices[i + 1]
            delta = model.addVar(vtype=copt.COPT.BINARY)
            model.addConstr(X[idx_1, j] <= u * (1 - delta))
            model.addConstr(X[idx_2, j] <= u * delta)

In [256]:
dry_lands = [
    "A1",
    "A2",
    "A3",
    "A4",
    "A5",
    "A6",
    "B1",
    "B2",
    "B3",
    "B4",
    "B5",
    "B6",
    "B7",
    "B8",
    "B9",
    "B10",
    "B11",
    "B12",
    "B13",
    "B14",
    "C1",
    "C2",
    "C3",
    "C4",
    "C5",
    "C6",
]
wet_lands = [
    "D1",
    "D2",
    "D3",
    "D4",
    "D5",
    "D6",
    "D7",
    "D8",
    "E1",
    "E2",
    "E3",
    "E4",
    "E5",
    "E6",
    "E7",
    "E8",
    "E9",
    "E10",
    "E11",
    "E12",
    "E13",
    "E14",
    "E15",
    "E16",
    "F1",
    "F2",
    "F3",
    "F4",
]

In [257]:
# 约束6:三年豆类约束
for land in dry_lands:
    land_indices = result_full[result_full["地块名"] == land].index

    for i in range(0, 6):
        expr = copt.quicksum(
            X[idx, col] for idx in land_indices[i : i + 3] for col in range(0, 5)
        )  # sum over 0:5 columns
        model.addConstr(
            expr >= 0.2 * result_full["种植上限"][land_indices[0]],
            name=f"dry_land_{land}_constraint_{i}",
        )

for land in wet_lands:
    land_indices = result_full[result_full["地块名"] == land].index

    for i in range(0, 11):
        expr = copt.quicksum(
            X[idx, col]
            for idx in land_indices[i : i + 6]
            for col in range(16, 19)
        )
        model.addConstr(
            expr >= 0.2 * result_full["种植上限"][land_indices[0]],
            name=f"wet_land_{land}_constraint_{i}",
        )

In [258]:
yield_data = pd.read_csv("data/结果/full_亩产量.csv").iloc[:, 4:-1].values
cost_data = pd.read_csv("data/结果/full_种植成本.csv").iloc[:, 4:-1].values
price_data = pd.read_csv("data/结果/full_销售单价.csv").iloc[:, 4:-1].values
sale_data = pd.read_csv("data/结果/full_预期销售量.csv").iloc[:, 2:].values

In [259]:
# S:实际销售量矩阵
S = model.addVars(656, 41, vtype=copt.COPT.CONTINUOUS, lb=0)

model.addConstrs(
    S[i, j] <= X[i, j] * yield_data[i, j] for i in range(656) for j in range(41)
)

# 目标函数
objective = copt.quicksum(
    0.5 * (S[i, j] - sale_data[0, j]) * price_data[i, j]
    + sale_data[0, j] * price_data[i, j]
    - X[i, j] * cost_data[i, j]
    for i in result_full[result_full["季度"] == 1].index.tolist()
    for j in range(41)
) + copt.quicksum(
    0.5 * (S[i, j] - sale_data[1, j]) * price_data[i, j]
    + sale_data[1, j] * price_data[i, j]
    - X[i, j] * cost_data[i, j]
    for i in result_full[result_full["季度"] == 2].index.tolist()
    for j in range(41)
)

In [260]:
# 求解
model.setObjective(objective, copt.COPT.MAXIMIZE)
model.solve()

if model.status == copt.COPT.OPTIMAL:
    optimized_X = np.zeros((656, 41))
    for i in range(656):
        for j in range(41):
            optimized_X[i, j] = X[i, j].x
else:
    print("未找到最优解！")

Model fingerprint: 44780f6c

Using Cardinal Optimizer v7.1.1 on Windows
Hardware has 12 cores and 16 threads. Using instruction set X86_AVX2 (10)
Maximizing a MIP problem

The original problem has:
    126062 rows, 79650 columns and 265230 non-zero elements
    52754 binaries

Starting the MIP solver with 16 threads and 32 tasks

Presolving the problem

The presolved problem has:
    42 rows, 38 columns and 221 non-zero elements
    38 binaries


     Nodes    Active  LPit/n  IntInf     BestBound  BestSolution    Gap   Time
         0         1      --       0  5.845719e+08            --    Inf  1.33s
H        0         1      --       0  5.845719e+08  5.845710e+08  0.00%  1.33s
         1         1     0.0       0  5.845719e+08  5.845710e+08  0.00%  1.33s
         1         1     0.0       0  5.845719e+08  5.845710e+08  0.00%  1.33s

Best solution   : 584570968.835500002
Best bound      : 584571860.207499266
Best gap        : 0.0002%
Solve time      : 1.34
Solve node      : 1
MIP stat

In [261]:
# 保存结果
result_full.iloc[:, 4:-1] = optimized_X.round(2)
result_full.to_csv("data/结果/result_full_case2.csv", index=False, encoding="utf-8-sig")
result_full

,年份,季度,地块名,地块id,黄豆,黑豆,红豆,绿豆,爬豆,小麦,...,黄心菜,芹菜,大白菜,白萝卜,红萝卜,榆黄菇,香菇,白灵菇,羊肚菌,种植上限
0,2023,1,A1,1,0.0,0.0,0.0,0.0,0.0,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
1,2023,1,A2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0
2,2023,1,A3,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0
3,2023,1,A4,4,72.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0
4,2023,1,A5,5,0.0,0.0,0.0,68.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,2030,2,E16,652,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
652,2030,2,F1,653,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
653,2030,2,F2,654,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
654,2030,2,F3,655,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6


In [262]:
result_full[result_full["地块名"]=='A1']

,年份,季度,地块名,地块id,黄豆,黑豆,红豆,绿豆,爬豆,小麦,...,黄心菜,芹菜,大白菜,白萝卜,红萝卜,榆黄菇,香菇,白灵菇,羊肚菌,种植上限
0,2023,1,A1,1,0.0,0.0,0.0,0.0,0.0,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
82,2024,1,A1,83,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
164,2025,1,A1,165,0.0,-0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
246,2026,1,A1,247,-0.0,0.0,0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
328,2027,1,A1,329,-0.0,-0.0,0.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
410,2028,1,A1,411,0.0,-0.0,16.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
492,2029,1,A1,493,0.0,-0.0,-0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
574,2030,1,A1,575,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
